In [1]:
import pandas as pd
from collections import deque

In [2]:
df = pd.read_csv('/Users/wouter/Documents/Finance/crypto/MEXC_Tickers.csv', sep=';')
df.head(200)

,USDT,USDC,ETH,BTC,TUSD
0,BTC,BTC,BTC,NaN,BTC
1,ETH,ETH,NaN,ETH,ETH
2,USDC,NaN,USDC,USDC,NaN
3,NaN,USDT,USDT,USDT,USDT
4,MX,MX,MX,MX,NaN
...,...,...,...,...,...
90,HOP,HOP,NaN,NaN,NaN
91,SHIT,SHIT,NaN,NaN,NaN
92,OLE,OLE,NaN,NaN,NaN
93,CCT,CCT,NaN,NaN,NaN


In [3]:
class Matrix:
    def __init__(self, df):
        # Keep the original DataFrame for reference
        self.headers = df.columns.tolist()
        self.df = df.reset_index(drop=True)
        self.matrix = self._convert_to_matrix(self.df.copy())
    
    def _convert_to_matrix(self, df):
        df = df.fillna(0) # Replace NaN with -1
        matrix = df.map(lambda x: 1 if isinstance(x, str) else x).values # Replace all strings with 1
        return matrix

    def print_matrix(self):
        print(self.matrix)
    
    def get_matrix(self):
        return self.matrix
    
    def get_df(self):
        return self.df
    
    def print_df(self):
        print(self.df)
        
    def get_currency(self, paths, row, col):
        # Get the base currency from the row index
        base_currency = self.df.iloc[row,col]  # Adjust as necessary if row index is not the currency
        
        # Get the quote currency from the column header
        quote_currency = self.headers[col]  # Adjust as necessary if column headers are not the currency
        
        return f"{base_currency}/{quote_currency}"

        
    def get_arbitrage_paths(self, start_row, start_col, max_length, conversion=True):
        all_paths = []
        
        # Queue stores (current_position, path, move count, last_move_was_horizontal, path_visited)
        queue = deque([((start_row, start_col), [(start_row, start_col)], 0, False, set([(start_row, start_col)]))]) 
        
        while queue:
            (row, col), path, moves_count, last_move_was_horizontal, path_visited = queue.popleft()
            
            # Debugging print
            
            if moves_count == max_length:
                if col == start_col:
                    all_paths.append(path)
                continue  # Stop exploring further for this path
            
            if last_move_was_horizontal:
                moves = self.get_vertical_moves(row, col)
            else:
                moves = self.get_horizontal_moves(row, col)
            
            for move in moves:
                new_row, new_col = move
                if (new_row, new_col) not in path_visited:
                    new_path = path + [move]
                    new_path_visited = path_visited.copy()
                    new_path_visited.add((new_row, new_col))
                    queue.append(((new_row, new_col), new_path, moves_count + 1, not last_move_was_horizontal, new_path_visited))
        
        if conversion:
            currency_paths = [
            [self.get_currency(all_paths, row, col) for row, col in path]
            for path in all_paths]
            
        else:
            currency_paths = all_paths
        
        return currency_paths
    
    def get_horizontal_moves(self, row, col):
        cols = self.matrix.shape[1]
        current = (row, col)
        moves = []
        for i in range(cols):
            if self.matrix[row][i] == 1 and (row, i) != current:
                moves.append((row, i))
        return moves
    
    def get_vertical_moves(self, row, col):
        rows = self.matrix.shape[0]
        current = (row, col)
        moves = []
        for i in range(rows):
            if self.matrix[i][col] == 1 and (i, col) != current:
                moves.append((i, col))
        return moves

In [4]:
MEXC = Matrix(df)
results = MEXC.get_arbitrage_paths(18, 3, 3, conversion=True)

print("Traversal Results:")
for path in results:
    print(path)

Traversal Results:
['EOS/BTC', 'EOS/USDT', 'ETH/USDT', 'ETH/BTC']
['EOS/BTC', 'EOS/USDT', 'USDC/USDT', 'USDC/BTC']
['EOS/BTC', 'EOS/USDT', 'MX/USDT', 'MX/BTC']
['EOS/BTC', 'EOS/USDT', 'XRP/USDT', 'XRP/BTC']
['EOS/BTC', 'EOS/USDT', 'ELF/USDT', 'ELF/BTC']
['EOS/BTC', 'EOS/USDT', 'DEXE/USDT', 'DEXE/BTC']
['EOS/BTC', 'EOS/USDT', 'ANKR/USDT', 'ANKR/BTC']
['EOS/BTC', 'EOS/USDT', 'SENSO/USDT', 'SENSO/BTC']
['EOS/BTC', 'EOS/USDT', 'TRX/USDT', 'TRX/BTC']
['EOS/BTC', 'EOS/USDT', 'ATOM/USDT', 'ATOM/BTC']
['EOS/BTC', 'EOS/USDT', 'BCH/USDT', 'BCH/BTC']
['EOS/BTC', 'EOS/USDT', 'SOL/USDT', 'SOL/BTC']
['EOS/BTC', 'EOS/USDT', 'LTC/USDT', 'LTC/BTC']
['EOS/BTC', 'EOS/USDT', 'ADA/USDT', 'ADA/BTC']
['EOS/BTC', 'EOS/USDT', 'DOT/USDT', 'DOT/BTC']
['EOS/BTC', 'EOS/USDT', 'PCX/USDT', 'PCX/BTC']
['EOS/BTC', 'EOS/USDT', 'KSM/USDT', 'KSM/BTC']
['EOS/BTC', 'EOS/USDT', 'DASH/USDT', 'DASH/BTC']
['EOS/BTC', 'EOS/USDT', 'IRIS/USDT', 'IRIS/BTC']
['EOS/BTC', 'EOS/USDT', 'LBTC/USDT', 'LBTC/BTC']
['EOS/BTC', 'EOS/USDT', '